In [1]:
!pip install requests beautifulsoup4 psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import psycopg2

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       # Ganti dengan nama database Anda
    user="postgres",         # Ganti dengan username PostgreSQL Anda
    password="felixexe131105",     # Ganti dengan password PostgreSQL Anda
    host="localhost",         # Jika PostgreSQL berjalan di komputer lokal
    port="5432"                   # Port default PostgreSQL
)

cursor = connection.cursor()
print("Koneksi berhasil!")


Koneksi berhasil!


In [4]:
import psycopg2
import requests
from bs4 import BeautifulSoup

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       
    user="postgres",             
    password="felixexe131105",   
    host="localhost",            # Ganti dengan host jika berbeda
    port="5432"                  # Port default PostgreSQL
)

cursor = connection.cursor()

# URL dari halaman e-commerce (contoh halaman produk)
url = "https://www.tokopedia.com/search?st=&q=HANDPHONE&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Lakukan HTTP GET request
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parse konten halaman menggunakan BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Cari elemen HTML yang berisi nama dan harga produk
produk = []
for item in soup.find_all("div", class_="_6+OpBPVGAgqnmycna+bWIw=="):  # Sesuaikan selector berdasarkan situs
    nama = item.find("span", class_="_0T8-iGxMpV6NEsYEhwkqEg==").get_text()  # Selector untuk nama produk
    
    # Cari harga produk
    harga_item = item.find_next("div", class_="_67d6E1xDKIzw+i2D2L0tjw== t4jWW3NandT5hvCFAiotYg==")
    if harga_item:
        harga = harga_item.get_text().strip()  # Ambil harga dan hilangkan spasi
        harga = int(harga.replace("Rp", "").replace(".", ""))  # Konversi harga ke integer (hapus "Rp" dan titik)
        produk.append((nama, harga))

# Masukkan data ke tabel ProdukEcommerce
for p in produk:
    try:
        # Query untuk memasukkan data ke dalam tabel
        query = "INSERT INTO ProdukEcommerce (nama_produk, harga) VALUES (%s, %s)"
        cursor.execute(query, (p[0], p[1]))  # Eksekusi query dengan data nama dan harga produk
        print(f"Data berhasil dimasukkan: {p[0]} | Harga: {p[1]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit perubahan ke database
connection.commit()

# Tutup koneksi
cursor.close()
connection.close()


Data berhasil dimasukkan: Samsung Galaxy S23 [8/256], Smartphone AI, AI Phone, Handphone AI, 50MP Nightography Camera & Snapdragon 8 Gen 2 (4nm), HP flagship Samsung, Android | Harga: 8999500
Data berhasil dimasukkan: vivo Y100 (8/256) - Magical Color Changing, 80W FlashCharge, AMOLED - Ungu Anggrek | Harga: 2449000
Data berhasil dimasukkan: Samsung Galaxy Z Fold 6 Fold6 12/512 12/256GB [ Free Upgrade Memory ] Silver Shadow Navy Pink Original | Harga: 24499000
Data berhasil dimasukkan: Samsung Galaxy Z Flip 6 Flip6 12/512 12/256GB Blue Mint Silver Shadow Yellow Original (Harga Sebelum Diskon) | Harga: 15999000
Data berhasil dimasukkan: Realme C75 8GB 256GB | 8GB 128GB | Dust & Water Resistance| Military Grade Shock Resistance | 6000mAh Massive Battery | Harga: 2425000
Data berhasil dimasukkan: Xiaomi Poco X5 Pro 5G 8/256GB 6/128GB Garansi Resmi Poco X5 Pro 5G | Harga: 2425000
Data berhasil dimasukkan: Infinix GT 20 Pro 5G 12/256GB - Up to 24GB Extended RAM - Dimensity 8200 Ultimate - 6